In [1]:
    #from sklearn.model_selection import train_test_split
#from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree

#from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
#from sklearn.metrics import PredictionErrorDisplay

from sklearn.decomposition import PCA

#from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import datetime
import seaborn as sns
sns.set_palette('muted')
sns.set_color_codes('muted')
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')

from statsmodels.graphics.mosaicplot import mosaic
pd.set_option('display.max_columns', 70)

**downloard the data**

In [2]:
df_geo = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_features_spb.csv')
df_time = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_time_based_features_full.csv')
df_hex_unique = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_hex_unique_with_shares_spb.csv')
df_hex_total = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_hex_total_with_shares_spb.csv')

In [3]:
df_geo.shape, df_time.shape, df_hex_unique.shape, df_hex_total.shape

((3905, 43), (3892, 63), (3892, 68), (3892, 68))

In [4]:
df_geo.total_fit_clubs.sum()

582.0

In [5]:
df_geo[['h3_9', 'total_fit_clubs']]

,h3_9,total_fit_clubs
0,8911060006bffff,0.0
1,89110600077ffff,0.0
2,89110600113ffff,4.0
3,89110600127ffff,0.0
4,89110600183ffff,0.0
...,...,...
3900,891106b9e4fffff,0.0
3901,891106b9e6bffff,0.0
3902,891106b9e6fffff,0.0
3903,891106b9e77ffff,0.0


In [6]:
import h3
def calculate_fit_club_sums(hex_id, df):
    neighbors = h3.k_ring(hex_id, 1)
    # Including the hex itself in neighbors for the total_fit_clubs_neighbour_and_hex calculation
    total_neighbors = df[df['h3_9'].isin(neighbors)]['total_fit_clubs'].sum()
    # Exclude the hex itself for the total_fit_clubs_neighbour calculation
    neighbors.remove(hex_id)
    total_neighbors_excluding_hex = df[df['h3_9'].isin(neighbors)]['total_fit_clubs'].sum()
    return len(neighbors), total_neighbors_excluding_hex, total_neighbors

In [7]:
df_data_lens = df_geo[['h3_9', 'total_fit_clubs']].copy()

In [8]:
df_data_lens[['neighbour_amount', 'total_fit_clubs_neighbour', 'total_fit_clubs_neighbour_and_hex']] = \
    df_data_lens.apply(lambda row: calculate_fit_club_sums(row['h3_9'], df_data_lens), axis=1, result_type='expand')

In [9]:
df_data_lens.shape

(3905, 5)

In [10]:
df_data_lens.head(3)

,h3_9,total_fit_clubs,neighbour_amount,total_fit_clubs_neighbour,total_fit_clubs_neighbour_and_hex
0,8911060006bffff,0.0,6.0,0.0,0.0
1,89110600077ffff,0.0,6.0,0.0,0.0
2,89110600113ffff,4.0,6.0,0.0,4.0


In [11]:
df_data_lens.neighbour_amount.value_counts()

6.0    3905
Name: neighbour_amount, dtype: int64

In [12]:
df_data_lens.total_fit_clubs_neighbour_and_hex.sum()

3723.0

In [13]:
df_data_lens[['h3_9', 'total_fit_clubs', 'total_fit_clubs_neighbour_and_hex']]

,h3_9,total_fit_clubs,total_fit_clubs_neighbour_and_hex
0,8911060006bffff,0.0,0.0
1,89110600077ffff,0.0,0.0
2,89110600113ffff,4.0,4.0
3,89110600127ffff,0.0,0.0
4,89110600183ffff,0.0,0.0
...,...,...,...
3900,891106b9e4fffff,0.0,0.0
3901,891106b9e6bffff,0.0,0.0
3902,891106b9e6fffff,0.0,0.0
3903,891106b9e77ffff,0.0,0.0


In [14]:
df_data_lens.total_fit_clubs.value_counts()

0.0    3454
1.0     358
2.0      69
3.0      16
5.0       4
4.0       3
6.0       1
Name: total_fit_clubs, dtype: int64

In [15]:
df_data_lens.total_fit_clubs_neighbour_and_hex.value_counts()

0.0     2082
1.0      885
2.0      488
3.0      204
4.0      110
5.0       69
6.0       30
7.0       23
8.0        8
10.0       4
11.0       1
9.0        1
Name: total_fit_clubs_neighbour_and_hex, dtype: int64

**join of other features for datalens**

In [16]:
df_time.columns

Index(['h3_9', 'avg_duration_hrs', 'median_duration_hrs', 'weekday_visits',
       'weekend_visits', 'Stay0_05h', 'Stay05_1h', 'Stay1_3h', 'Stay3_6h',
       'Stay6_9h', 'Stay9_15h', 'Stay15_20h', 'Stay20_plus', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'day', 'evening', 'morning', 'night', 'weekday_visits_share',
       'weekend_visits_share', 'Stay0_05h_share', 'Stay05_1h_share',
       'Stay1_3h_share', 'Stay3_6h_share', 'Stay6_9h_share', 'Stay9_15h_share',
       'Stay15_20h_share', 'Stay20_plus_share', 'Friday_share', 'Monday_share',
       'Saturday_share', 'Sunday_share', 'Thursday_share', 'Tuesday_share',
       'Wednesday_share', 'day_share', 'evening_share', 'morning_share',
       'night_share', 'avg_duration_hrs_per_unique',
       'median_duration_hrs_per_unique', 'Stay0_05h_per_unique',
       'Stay05_1h_per_unique', 'Stay1_3h_per_unique', 'Stay3_6h_per_unique',
       'Stay6_9h_per_unique', 'Stay9_15h_per_unique', 'Stay15

In [17]:
df_combined = pd.merge(df_data_lens[['h3_9', 'total_fit_clubs', 'total_fit_clubs_neighbour_and_hex']],
                           df_hex_total[['h3_9', 'total_visits', 'salary_1_total_share', 'salary_2_total_share',
                                         'salary_3_total_share', 'salary_4_total_share', 'salary_5_total_share']],
                           how='left', left_on='h3_9', right_on='h3_9')

In [18]:
df_combined = pd.merge(df_combined,
                       df_hex_unique[['h3_9', 'unique_visits', 'salary_1_unique_share', 'salary_2_unique_share',
                                     'salary_3_unique_share', 'salary_4_unique_share', 'salary_5_unique_share']],
                        how='left', left_on='h3_9', right_on='h3_9')

In [19]:
df_combined = pd.merge(df_combined,
                       df_time[['h3_9', 'day_share', 'evening_share', 'morning_share', 'night_share']],
                        how='left', left_on='h3_9', right_on='h3_9')

In [20]:
df_combined.shape

(3905, 19)

In [21]:
df_combined.head(3)

,h3_9,total_fit_clubs,total_fit_clubs_neighbour_and_hex,total_visits,salary_1_total_share,salary_2_total_share,salary_3_total_share,salary_4_total_share,salary_5_total_share,unique_visits,salary_1_unique_share,salary_2_unique_share,salary_3_unique_share,salary_4_unique_share,salary_5_unique_share,day_share,evening_share,morning_share,night_share
0,8911060006bffff,0.0,0.0,147.0,0.248120,0.300752,0.203008,0.112782,0.135338,98.0,0.341176,0.176471,0.141176,0.152941,0.188235,0.319728,0.272109,0.190476,0.217687
1,89110600077ffff,0.0,0.0,250.0,0.144860,0.182243,0.233645,0.257009,0.182243,99.0,0.229885,0.149425,0.229885,0.206897,0.183908,0.460000,0.216000,0.144000,0.180000
2,89110600113ffff,4.0,4.0,333.0,0.215873,0.196825,0.206349,0.222222,0.158730,144.0,0.325581,0.201550,0.170543,0.155039,0.147287,0.444444,0.153153,0.252252,0.150150


In [25]:
df_combined.loc[df_combined.total_visits != df_combined.unique_visits][['h3_9', 'total_visits', 'unique_visits']]

,h3_9,total_visits,unique_visits
0,8911060006bffff,147.0,98.0
1,89110600077ffff,250.0,99.0
2,89110600113ffff,333.0,144.0
4,89110600183ffff,16.0,7.0
6,8911060018bffff,57.0,19.0
...,...,...,...
3899,891106b9e47ffff,31.0,15.0
3900,891106b9e4fffff,16.0,12.0
3901,891106b9e6bffff,36.0,9.0
3902,891106b9e6fffff,18.0,7.0


In [30]:
path = 'C:/Users/elysi/Tele2_project/data/pre_model_state/df_data_lens_may.csv'
df_combined.to_csv(path, index=False)

In [26]:
df_lens = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_data_lens_may.csv')

In [27]:
df_lens.loc[df_lens.total_visits != df_lens.unique_visits][['h3_9', 'total_visits', 'unique_visits']]

,h3_9,total_visits,unique_visits
0,8911060006bffff,147.0,98.0
1,89110600077ffff,250.0,99.0
2,89110600113ffff,333.0,144.0
4,89110600183ffff,16.0,7.0
6,8911060018bffff,57.0,19.0
...,...,...,...
3899,891106b9e47ffff,31.0,15.0
3900,891106b9e4fffff,16.0,12.0
3901,891106b9e6bffff,36.0,9.0
3902,891106b9e6fffff,18.0,7.0


In [ ]:
path = 'C:/Users/elysi/Tele2_project/data/pre_model_state/df_data_lens_may.csv'
df_combined.to_csv(path, index=False)